In [1]:
%load_ext autoreload
%autoreload 2
%aimport

Modules to reload:
all-except-skipped

Modules to skip:



In [2]:
%matplotlib inline

In [3]:
from relaxed.plot_defaults import *

In [4]:
import astropy
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt 
import re 
from astropy.table import Table
import astropy.table
import json
from scipy import stats
from copy import deepcopy
import scipy
import warnings

import matplotlib as mpl
from sklearn.model_selection import train_test_split



In [6]:
from relaxed.analysis import setup, get_quantiles, get_am, get_an_from_am, get_ma, get_gradient, \
get_fractional_tdyn, get_savgol_grads, get_tt_indices, add_box_indices, vol_jacknife_err
from relaxed.models import training_suite
from relaxed import halo_catalogs

In [22]:
mah_data = setup('m11', "../../../output/subhalo", cutoff_missing=0.01, cutoff_particle=0.1, min_mass_bin=0.4)
cat = mah_data['cat']
scales = mah_data['scales']
indices = mah_data['indices'].astype(int)

In [10]:
# first we need merger ratio at every scale 
merger_ratio  = np.zeros((len(cat), len(scales)))
for i in range(len(cat)):
    for j, idx in enumerate(indices):
        f_sub_name = f'f_sub_a{idx}'
        m2_name = f'm2_a{idx}'
        cpgratio_name = f'cpgratio_a{idx}'
        merger_ratio[i, j] = np.nanmax([cat[m2_name][i], cat[cpgratio_name][i]])

['id',
 'upid',
 'mvir',
 'rvir',
 'rs',
 'xoff',
 'voff',
 'x',
 'y',
 'z',
 'x0',
 'v0',
 'cvir',
 'spin',
 'q',
 'vvir',
 't/|u|',
 'eta',
 'phi_l',
 'gamma_tdyn',
 'tdyn',
 'scale_of_last_mm',
 'f_sub_a0',
 'f_sub_a1',
 'f_sub_a2',
 'f_sub_a3',
 'f_sub_a4',
 'f_sub_a5',
 'f_sub_a6',
 'f_sub_a7',
 'f_sub_a8',
 'f_sub_a9',
 'f_sub_a10',
 'f_sub_a11',
 'f_sub_a12',
 'f_sub_a13',
 'f_sub_a14',
 'f_sub_a15',
 'f_sub_a16',
 'f_sub_a17',
 'f_sub_a18',
 'f_sub_a19',
 'f_sub_a20',
 'f_sub_a21',
 'f_sub_a22',
 'f_sub_a23',
 'f_sub_a24',
 'f_sub_a25',
 'f_sub_a26',
 'f_sub_a27',
 'f_sub_a28',
 'f_sub_a29',
 'f_sub_a30',
 'f_sub_a31',
 'f_sub_a32',
 'f_sub_a33',
 'f_sub_a34',
 'f_sub_a35',
 'f_sub_a36',
 'f_sub_a37',
 'f_sub_a38',
 'f_sub_a39',
 'f_sub_a40',
 'f_sub_a41',
 'f_sub_a42',
 'f_sub_a43',
 'f_sub_a44',
 'f_sub_a45',
 'f_sub_a46',
 'f_sub_a47',
 'f_sub_a48',
 'f_sub_a49',
 'f_sub_a50',
 'f_sub_a51',
 'f_sub_a52',
 'f_sub_a53',
 'f_sub_a54',
 'f_sub_a55',
 'f_sub_a56',
 'f_sub_a57',
 

In [19]:
# now construct scale of last MM (at present day) as a function fo different thresholds
Mu = np.linspace(0.1, 1, num=10)
scale_of_last_MM = np.zeros((len(cat), len(Mu)))
for i in range(len(cat)):
    for k, mu in enumerate(Mu):
        MM_indices = np.where(merger_ratio[i] > mu)[0]
        if len(MM_indices) > 0: 
            scale_index = MM_indices[-1] # merger ratio (like everything else) is ordered from early -> late, so take last one which is most recent one.
            scale_of_last_MM[i, k] = scales[scale_index]
        else: 
            scale_of_last_MM[i, k] = np.nan